## Lab6 UCM EN Demo

In [1]:
import os

In [2]:
# Enalbe 780M with ROCm
os.environ['HSA_OVERRIDE_GFX_VERSION'] = '11.0.0'

print(os.environ['HSA_OVERRIDE_GFX_VERSION'])

In [3]:
import torch

In [4]:
print(torch.__version__)

2.3.0+rocm6.0


In [5]:
# Query GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    print('Using GPU:', torch.cuda.get_device_name(0))
    print('GPU properties:', torch.cuda.get_device_properties(0))
else:
    device = torch.device("cpu")
    print('Using CPU')

Using GPU: AMD Radeon Graphics
GPU properties: _CudaDeviceProperties(name='AMD Radeon Graphics', major=11, minor=0, gcnArchName='gfx1100', total_memory=16384MB, multi_processor_count=6)


In [6]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.

import nest_asyncio

nest_asyncio.apply()

In [7]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.node_parser import SentenceSplitter

from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

# Set embedding model
# Please download it ahead running this lab by "ollama pull nomic-embed-text"
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")

# Set ollama model
Settings.llm = Ollama(model="llama3", request_timeout=200.0)

/home/igpu/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [8]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

if not os.path.exists("./chroma_db/CUM_en_db"):
    # initialize client
    db = chromadb.PersistentClient(path="./chroma_db/CUM_en_db")
    # get collection
    chroma_collection = db.get_or_create_collection("CUM_en_db")
    # assign chroma as the vector_store to the context
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    # Load data
    documents = SimpleDirectoryReader(input_files=["../data/FordUM.pdf"]).load_data()
    # Build vector index per-document
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        transformations=[SentenceSplitter(chunk_size=1024, chunk_overlap=50)],
    )
else:
    # initialize client
    db = chromadb.PersistentClient(path="./chroma_db/CUM_en_db")
    # get collection
    chroma_collection = db.get_or_create_collection("CUM_en_db")
    # assign chroma as the vector_store to the context
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    # load your index from stored vectors
    index = VectorStoreIndex.from_vector_store(
        vector_store, storage_context=storage_context
    )

In [9]:
# create a query engine
query_engine = index.as_query_engine(streaming=True, response_mode="compact", similarity_top_k=3)

In [10]:
# Query Test 0
response = query_engine.query("Which pages could get the detials about Exterior Lighting Control?")
response.print_response_stream()

 on the provided context information, I would suggest that pages 127-128 or 365-366 might have details about Exterior Lighting Control. However, since the exact page numbers are not explicitly mentioned in the given context, it's possible that the relevant information could be spread across multiple pages or even on a different document altogether.

In [11]:
# Q
response = query_engine.query("find out the phone number of the customer relationship center in China from the Car User Manual")
response.print_response_stream()

'm happy to help! However, I must point out that the provided context information does not contain any phone numbers or details related to the customer relationship center in China. The context appears to be focused on the features and controls of a Ford vehicle, specifically the F-150 Lightning.

As such, I cannot provide an answer to your query based on the given context information. If you could provide more relevant context or clarify what you are looking for, I'll do my best to assist you.

In [12]:
# Q
response = query_engine.query("What's the steps to SWITCHING THE DAYTIME RUNNING LAMPS ON AND OFF and which pages show it?")
response.print_response_stream()

 to the provided context, the steps to switch the daytime running lamps on and off are:

1. Press Settings on the touch screen.
2. Press Vehicle Settings.
3. Press Lighting.
4. Switch Daytime Running Lights on or off.

This information can be found on pages 115 and 127 of the Ford F-150 Lightning manual.

## To-Do: Optimization the RAG

Refer to Basic Strategies: https://docs.llamaindex.ai/en/v0.10.19/optimizing/basic_strategies/basic_strategies.html